In [14]:
import pandas as pd

In [15]:
two_year_return = pd.read_excel('./data/under_value.xlsx')

In [16]:
# g_result = pd.read_excel('./data/g_result.xlsx')
# two_year_return.sort_values('date_indicator', inplace=True)
# g_result.sort_values('DT',inplace=True)
# two_year_return = pd.merge_asof(two_year_return, g_result, left_by='stock_code', right_by='SEC_ID', left_on='date_indicator', right_on='DT')

In [17]:
two_year_return.columns

Index(['industry_name_1', 'industry_name_2', 'stock_code', 'stock_name',
       'year', 'month', 'industry_indicator', 'date_indicator',
       'indicator_value', 'indicator_percent', 'pred_indicator_value',
       'pred_indicator_percent', 'median_indicator', 'mean_indicator',
       'pred_indicator', 'cagr', 'median_pred_return', 'mean_pred_return',
       'price_industry_indicator_corr'],
      dtype='object')

In [18]:
universe_stocks = pd.read_excel('./data/capex.xlsx')

In [19]:
# universe_stocks.columns

In [20]:
# two_year_return = pd.merge(two_year_return, universe_stocks.drop(columns=['stock_name', 'industry_name_1', 'industry_name_2']), on=['stock_code', 'year', 'month'], how='inner')

In [21]:
two_year_return['stock_code'] = two_year_return['stock_code'].apply(lambda x : x[:-3]+'.XSHG' if x[0]=='6' else x[:-3]+'.XSHE')

In [22]:
two_year_return.columns

Index(['industry_name_1', 'industry_name_2', 'stock_code', 'stock_name',
       'year', 'month', 'industry_indicator', 'date_indicator',
       'indicator_value', 'indicator_percent', 'pred_indicator_value',
       'pred_indicator_percent', 'median_indicator', 'mean_indicator',
       'pred_indicator', 'cagr', 'median_pred_return', 'mean_pred_return',
       'price_industry_indicator_corr'],
      dtype='object')

In [23]:
target_stocks = pd.DataFrame(columns=['date','industry_1','industry_2','stock_code','stock_name',
                                      'indicator','indicator_percent','pred_indicator_percent', 'cagr'])

In [24]:
%%backtest
#引入Backtest

target_stocks = target_stocks.drop(target_stocks.index)

def get_target_comp_return(df):
    df = df.copy()
    if len(df) >= 0:
        df.sort_values('median_pred_return', ascending=False, inplace=True)
        return df.iloc[:int(len(df)/3)]
    
def get_target_comp_capex(df):
    df = df.copy()
    if len(df) >= 0:
        df.sort_values('rank_sum', ascending=True, inplace=True)
        return df.iloc[:int(len(df)/2)]
    
import quartz_extensions.Backtest as btr

start = '2016-05-01'
end = '2021-04-18'

benchmark = 'HS300'
universe = StockUniverse('HS300') # 证券池，支持股票和基金、期货
                  
freq = 'd'
refresh_rate = Monthly(1)

accounts = {
    'stock_account': AccountConfig(account_type='security', capital_base=10000000)
}

# 把回测参数封装到SimulationParameters中，供Backtest使用
sim_params = btr.SimulationParameters(start, end, benchmark, universe, refresh_rate=refresh_rate, accounts=accounts, freq=freq)

# ---------------回测参数部分结束----------------
# 获取回测行情数据
data = btr.get_backtest_data(sim_params)

# 运行结果
results = {}

# -----策略定义开始，这和常用的策略编写模式完全一样-----    
def initialize(context):
    pass

def handle_data(context):
    
    if context.current_date.month != 5:
        return None
    current_year = context.current_date.year
    
    all_universe = context.get_universe('stock', exclude_halt=True)
    current_universe = all_universe.copy()
    print('\n', context.current_date)
    
    current_two_year_return = two_year_return[(two_year_return['year'] == current_year) &\
                                              (two_year_return['month'] == context.current_date.month) &\
                                              (two_year_return['stock_code'].isin(current_universe))].copy()
    print('stock_num: ', current_two_year_return.shape[0], end=' -> ')
    
    # 低估值
    current_two_year_return_1 = current_two_year_return.groupby('industry_name_1', as_index=False, group_keys=False).apply(get_target_comp_return)
    print(current_two_year_return_1.shape[0], end=' -> ')
    current_two_year_return_1 = current_two_year_return_1[current_two_year_return_1['price_industry_indicator_corr']>=0.6]
    print(current_two_year_return_1.shape[0], end=' -> ')
#     current_two_year_return_1 = current_two_year_return_1[current_two_year_return_1['cagr']>=0.1]
#     print(current_two_year_return_1.shape[0], end=' -> ')
    current_two_year_return_1 = current_two_year_return_1[current_two_year_return_1['median_pred_return']>=10]
    print(current_two_year_return_1.shape[0])
    
    # 扩产能
#     current_two_year_return_2 = current_two_year_return.groupby('industry_name_1', as_index=False, group_keys=False).apply(get_target_comp_capex)
#     current_two_year_return_2 = current_two_year_return_2[current_two_year_return_2['mean_corr']>=0.5]
    
#     print(current_two_year_return_2.shape[0], end=' -> ')
    
#     current_two_year_return_1 = current_two_year_return_1[current_two_year_return_1['Governance_3month_mean_rank_label'].isin(['相对出色','相对较好'])]
#     current_two_year_return_2 = current_two_year_return_2[current_two_year_return_2['Governance_3month_mean_rank_label'].isin(['相对出色','相对较好'])]
#     print(current_two_year_return_1.shape[0], '∩', current_two_year_return_2.shape[0], end=' -> ')

#     target_stock_code_list = set(current_two_year_return_1['stock_code'].tolist()).intersection(set(current_two_year_return_2['stock_code'].tolist()))
#     target_stock_name_list = set(current_two_year_return_1['stock_name'].tolist()).intersection(set(current_two_year_return_2['stock_name'].tolist()))

    target_stock_code_list = set(current_two_year_return_1['stock_code'].tolist())
    target_stock_name_list = set(current_two_year_return_1['stock_name'].tolist())
#     print(len(target_stock_code_list))
    
    global target_stocks
    for stock_code in target_stock_code_list:
        single_stock = current_two_year_return[current_two_year_return['stock_code'] == stock_code]
        target_stocks = target_stocks.append({'date': context.current_date,
                                              'industry_1': single_stock['industry_name_1'].values[0],
                                              'industry_2': single_stock['industry_name_2'].values[0],
                                              'stock_code': single_stock['stock_code'].values[0],
                                              'stock_name': single_stock['stock_name'].values[0],
                                              'indicator': single_stock['industry_indicator'].values[0],
                                              'indicator_percent': single_stock['indicator_percent'].values[0],
                                              'pred_indicator_percent': single_stock['pred_indicator_percent'].values[0],
                                              'cagr': single_stock['cagr'].values[0]},
                                             ignore_index=True, 
                                             sort=True)
    
    target_num = 40
    
    print('本期target的股票列表')
    print(target_stock_name_list)
    
    print('开始账户操作')
    
    account = context.get_account('stock_account')
        
    current_position = account.get_positions(exclude_halt=True)
    print('当前持仓可交易')
    print(set(current_position))
    print('当前持仓全部包括')
    print(set(account.get_positions(exclude_halt=False)))
    
     
    for stock in set(current_position).difference(target_stock_code_list):
        account.order_to(stock, 0)  
        
#     previous_date = context.previous_date.strftime('%Y-%m-%d')
    # 获取市值的的历史数据集，截止到前一个交易日
#     hist = context.history(symbol=target_stock_code_list, attribute='MktValue', time_range=1, style='tas')[previous_date]
    
    for stock in target_stock_code_list:
        # 根据市值下单
#         account.order_pct_to(stock,  hist['MktValue'].loc[stock] / hist['MktValue'].sum())
        account.order_pct_to(stock,  1 / len(target_stock_code_list))
        
# 生成策略对象
strategy = btr.TradingStrategy(initialize, handle_data)

# -------------------策略定义结束--------------------   

# 开始回测
bt, perf = btr.backtest(sim_params, strategy, data=data)

print('\n策略年化收益率：', format(perf['annualized_return'],'.2%'))
print('基准年化收益率：', format(perf['benchmark_annualized_return'],'.2%'))
print('策略累计收益率：', format(perf['cumulative_returns'].values[-1],'.2%'))
print('基准累计收益率：', format(perf['benchmark_cumulative_returns'].values[-1],'.2%'))

target_stocks = target_stocks[['date','industry_1','industry_2','stock_code','stock_name',
                               'indicator','indicator_percent','pred_indicator_percent', 'cagr']]
target_stocks.to_excel('target_stocks_5.xlsx')

Chart()


 2016-05-03 00:00:00
stock_num:  128 -> 34 -> 24 -> 24
本期target的股票列表
{'广联达', '招商证券', '国投电力', '中国银行', '浙江龙盛', '农业银行', '华夏幸福', '荣盛发展', '中国平安', '复星医药', '大华股份', '金风科技', '立讯精密', '云南白药', '光大银行', '建设银行', '伊利股份', '长安汽车', '国金证券', '中国建筑', '白云山', '上海莱士', '海康威视', '工商银行'}
开始账户操作
当前持仓可交易
set()
当前持仓全部包括
set()

 2017-05-02 00:00:00
stock_num:  140 -> 38 -> 19 -> 19
本期target的股票列表
{'华兰生物', '中信银行', '爱尔眼科', '华夏银行', '国投电力', '平安银行', '万华化学', '民生银行', '华夏幸福', '恒瑞医药', '金风科技', '立讯精密', '同花顺', '国投资本', '阳光城', '伊利股份', '葛洲坝', '西南证券', '浦发银行'}
开始账户操作
当前持仓可交易
{'600340.XSHG', '002475.XSHE', '002202.XSHE', '000538.XSHE', '600352.XSHG', '601668.XSHG', '002410.XSHE', '601939.XSHG', '601818.XSHG', '601318.XSHG', '002146.XSHE', '002236.XSHE', '600887.XSHG', '002415.XSHE', '000625.XSHE', '600109.XSHG', '601288.XSHG', '600196.XSHG', '600999.XSHG', '601398.XSHG', '600886.XSHG', '601988.XSHG', '600332.XSHG'}
当前持仓全部包括
{'600340.XSHG', '002475.XSHE', '002202.XSHE', '000538.XSHE', '600352.XSHG', '601668.XSHG', '002410.XSHE', '601939

In [26]:
%%backtest
#引入Backtest

target_stocks = target_stocks.drop(target_stocks.index)

def get_target_comp_return(df):
    df = df.copy()
    if len(df) >= 0:
        df.sort_values('median_pred_return', ascending=False, inplace=True)
        return df.iloc[:int(len(df)/3)]
    
def get_target_comp_capex(df):
    df = df.copy()
    if len(df) >= 0:
        df.sort_values('rank_sum', ascending=True, inplace=True)
        return df.iloc[:int(len(df)/2)]
    
import quartz_extensions.Backtest as btr

start = '2016-11-01'
end = '2021-04-18'

benchmark = 'HS300'
universe = StockUniverse('HS300') # 证券池，支持股票和基金、期货
                  
freq = 'd'
refresh_rate = Monthly(1)

accounts = {
    'stock_account': AccountConfig(account_type='security', capital_base=10000000)
}

# 把回测参数封装到SimulationParameters中，供Backtest使用
sim_params = btr.SimulationParameters(start, end, benchmark, universe, refresh_rate=refresh_rate, accounts=accounts, freq=freq)

# ---------------回测参数部分结束----------------
# 获取回测行情数据
data = btr.get_backtest_data(sim_params)

# 运行结果
results = {}

# -----策略定义开始，这和常用的策略编写模式完全一样-----    
def initialize(context):
    pass

def handle_data(context):
    
    if context.current_date.month != 11:
        return None
    current_year = context.current_date.year
    
    all_universe = context.get_universe('stock', exclude_halt=True)
    current_universe = all_universe.copy()
    print('\n', context.current_date)
    
    current_two_year_return = two_year_return[(two_year_return['year'] == current_year) &\
                                              (two_year_return['month'] == context.current_date.month) &\
                                              (two_year_return['stock_code'].isin(current_universe))].copy()
    print('stock_num: ', current_two_year_return.shape[0], end=' -> ')
    
    # 低估值
    current_two_year_return_1 = current_two_year_return.groupby('industry_name_1', as_index=False, group_keys=False).apply(get_target_comp_return)
    print(current_two_year_return_1.shape[0], end=' -> ')
    current_two_year_return_1 = current_two_year_return_1[current_two_year_return_1['price_industry_indicator_corr']>=0.6]
    print(current_two_year_return_1.shape[0], end=' -> ')
#     current_two_year_return_1 = current_two_year_return_1[current_two_year_return_1['cagr']>=0.1]
#     print(current_two_year_return_1.shape[0], end=' -> ')
    current_two_year_return_1 = current_two_year_return_1[current_two_year_return_1['median_pred_return']>=10]
    print(current_two_year_return_1.shape[0])
    
    # 扩产能
#     current_two_year_return_2 = current_two_year_return.groupby('industry_name_1', as_index=False, group_keys=False).apply(get_target_comp_capex)
#     current_two_year_return_2 = current_two_year_return_2[current_two_year_return_2['mean_corr']>=0.5]
    
#     print(current_two_year_return_2.shape[0], end=' -> ')
    
#     current_two_year_return_1 = current_two_year_return_1[current_two_year_return_1['Governance_3month_mean_rank_label'].isin(['相对出色','相对较好'])]
#     current_two_year_return_2 = current_two_year_return_2[current_two_year_return_2['Governance_3month_mean_rank_label'].isin(['相对出色','相对较好'])]
#     print(current_two_year_return_1.shape[0], '∩', current_two_year_return_2.shape[0], end=' -> ')

#     target_stock_code_list = set(current_two_year_return_1['stock_code'].tolist()).intersection(set(current_two_year_return_2['stock_code'].tolist()))
#     target_stock_name_list = set(current_two_year_return_1['stock_name'].tolist()).intersection(set(current_two_year_return_2['stock_name'].tolist()))

    target_stock_code_list = set(current_two_year_return_1['stock_code'].tolist())
    target_stock_name_list = set(current_two_year_return_1['stock_name'].tolist())
#     print(len(target_stock_code_list))
    
    global target_stocks
    for stock_code in target_stock_code_list:
        single_stock = current_two_year_return[current_two_year_return['stock_code'] == stock_code]
        target_stocks = target_stocks.append({'date': context.current_date,
                                              'industry_1': single_stock['industry_name_1'].values[0],
                                              'industry_2': single_stock['industry_name_2'].values[0],
                                              'stock_code': single_stock['stock_code'].values[0],
                                              'stock_name': single_stock['stock_name'].values[0],
                                              'indicator': single_stock['industry_indicator'].values[0],
                                              'indicator_percent': single_stock['indicator_percent'].values[0],
                                              'pred_indicator_percent': single_stock['pred_indicator_percent'].values[0],
                                              'cagr': single_stock['cagr'].values[0]},
                                             ignore_index=True, 
                                             sort=True)
    
    target_num = 40
    
    print('本期target的股票列表')
    print(target_stock_name_list)
    
    print('开始账户操作')
    
    account = context.get_account('stock_account')
        
    current_position = account.get_positions(exclude_halt=True)
    print('当前持仓可交易')
    print(set(current_position))
    print('当前持仓全部包括')
    print(set(account.get_positions(exclude_halt=False)))
    
     
    for stock in set(current_position).difference(target_stock_code_list):
        account.order_to(stock, 0)  
        
#     previous_date = context.previous_date.strftime('%Y-%m-%d')
    # 获取市值的的历史数据集，截止到前一个交易日
#     hist = context.history(symbol=target_stock_code_list, attribute='MktValue', time_range=1, style='tas')[previous_date]
    
    for stock in target_stock_code_list:
        # 根据市值下单
#         account.order_pct_to(stock,  hist['MktValue'].loc[stock] / hist['MktValue'].sum())
        account.order_pct_to(stock,  1 / len(target_stock_code_list))
        
# 生成策略对象
strategy = btr.TradingStrategy(initialize, handle_data)

# -------------------策略定义结束--------------------   

# 开始回测
bt, perf = btr.backtest(sim_params, strategy, data=data)

print('\n策略年化收益率：', format(perf['annualized_return'],'.2%'))
print('基准年化收益率：', format(perf['benchmark_annualized_return'],'.2%'))
print('策略累计收益率：', format(perf['cumulative_returns'].values[-1],'.2%'))
print('基准累计收益率：', format(perf['benchmark_cumulative_returns'].values[-1],'.2%'))

target_stocks = target_stocks[['date','industry_1','industry_2','stock_code','stock_name',
                               'indicator','indicator_percent','pred_indicator_percent', 'cagr']]
target_stocks.to_excel('target_stocks_11.xlsx')

Chart()


 2016-11-01 00:00:00
stock_num:  135 -> 36 -> 22 -> 22
本期target的股票列表
{'科大讯飞', '爱尔眼科', '华夏银行', '北方稀土', '国投电力', '中国银行', '平安银行', '万华化学', '上海机场', '华夏幸福', '中国平安', '浪潮信息', '金风科技', '光大银行', '国投资本', '长安汽车', '中国铁建', '九州通', '上海莱士', '方正证券', '乐普医疗', '工商银行'}
开始账户操作
当前持仓可交易
set()
当前持仓全部包括
set()

 2017-11-01 00:00:00
stock_num:  151 -> 42 -> 20 -> 18
本期target的股票列表
{'西部证券', '中国交建', '华兰生物', '华东医药', '北京银行', '君正集团', '美年健康', '葛洲坝', '民生银行', '华夏幸福', '南京银行', '紫光国微', '老板电器', '国投资本', '华夏银行', '大华股份', '南方航空', '浦发银行'}
开始账户操作
当前持仓可交易
{'600340.XSHG', '002202.XSHE', '600309.XSHG', '600998.XSHG', '601901.XSHG', '300003.XSHE', '000001.XSHE', '600061.XSHG', '002230.XSHE', '601818.XSHG', '601318.XSHG', '601186.XSHG', '000977.XSHE', '000625.XSHE', '600111.XSHG', '300015.XSHE', '601398.XSHG', '600009.XSHG', '600886.XSHG', '601988.XSHG', '002252.XSHE', '600015.XSHG'}
当前持仓全部包括
{'600340.XSHG', '002202.XSHE', '600309.XSHG', '600998.XSHG', '601901.XSHG', '300003.XSHE', '000001.XSHE', '600061.XSHG', '002230.XSHE', '601818.XSHG'